In [11]:
import pandas as pd
from tqdm import tqdm
from gensim.models.phrases import Phrases, Phraser
from gensim.models.word2vec import LineSentence
import itertools as it
import os

In [2]:
# table with id,year,text,keyword columns
df = pd.read_csv('./Processed_data/year_text_keywords_affiliation.csv').drop('Unnamed: 0',axis=1)

/Users/shunling/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,Id,year,text,keywords,affiliation
0,24877594,2014.0,what we know and do not know about the canna...,cannabinoid receptor 2 ; endocannabinoid ; i...,[]
1,16889837,2006.0,allosteric agonist of 7tm receptor expand th...,NaN,glaxosmithkline
2,31068464,2019.0,mrgprx4 be a g protein - couple receptor act...,mrgprx4 ; bile acid ; cholestasis ; itch ; p...,[' the johns hopkins university']
3,30610192,2019.0,modulation of gpr39 a g - protein couple rec...,NaN,"[' oregon national primate research center, or..."
4,30289386,2018.0,simulation of spontaneous g protein activati...,g protein ; markov state model ; allostery ;...,[' washington university']


In [9]:
%%time
df['text'] = df['text'].apply(lambda x: x.replace('-',''))

CPU times: user 1.02 s, sys: 300 ms, total: 1.32 s
Wall time: 1.59 s


In [10]:
with open('./phrases/singlegram.txt','w') as f:
    for i in tqdm(range(df.shape[0])):
        f.write(df.iloc[i].text)
        f.write('\n')

100%|██████████| 323646/323646 [01:06<00:00, 4902.79it/s]


In [17]:
# from unigram to digram
unigrams = LineSentence('./phrases/singlegram.txt')
bigram_phrases = Phrases(unigrams) # train model
bigram_model = Phraser(bigram_phrases) # build model
with open('./phrases/bigrams.txt','w') as f:
    for sent in tqdm(unigrams):
        bigram_sent = ' '.join(bigram_model[sent]) # apply model
        f.write(bigram_sent)
        f.write('\n')
        
# from bigram to trigram
bigrams = LineSentence('./phrases/bigrams.txt')
trigram_phrases = Phrases(bigrams)
trigram_model = Phraser(trigram_phrases)
with open('./phrases/trigrams.txt','w') as f:
    for sent in tqdm(bigrams):
        tri_sent = ' '.join(trigram_model[sent])
        f.write(tri_sent)
        f.write('\n')
        
# from trigram to fourgram
trigrams = LineSentence('./phrases/trigrams.txt')
fourgram_phrases = Phrases(trigrams)
fourgram_model = Phraser(fourgram_phrases)
with open('./phrases/fourgrams.txt','w') as f:
    for sent in tqdm(trigrams):
        four_sent = ' '.join(fourgram_model[sent])
        f.write(four_sent)
        f.write('\n')

323646it [03:07, 1725.19it/s]
323646it [02:51, 1892.36it/s]
323646it [02:53, 1865.31it/s]


In [20]:
text = []
f = open('./phrases/fourgrams.txt','r')
for line in tqdm(f.readlines()):
    text.append(line)

100%|██████████| 323646/323646 [00:00<00:00, 1867364.90it/s]


In [22]:
df['fourgram'] = text

In [23]:
df.head()

,Id,year,text,keywords,affiliation,fourgram
0,24877594,2014.0,what we know and do not know about the canna...,cannabinoid receptor 2 ; endocannabinoid ; i...,[],what we know and do_not know_about the cannabi...
1,16889837,2006.0,allosteric agonist of 7tm receptor expand th...,NaN,glaxosmithkline,allosteric agonist of 7tm receptor expand the ...
2,31068464,2019.0,mrgprx4 be a g protein couple receptor acti...,mrgprx4 ; bile acid ; cholestasis ; itch ; p...,[' the johns hopkins university'],mrgprx4 be a g_protein_couple receptor activat...
3,30610192,2019.0,modulation of gpr39 a g protein couple rece...,NaN,"[' oregon national primate research center, or...",modulation of gpr39 a g_protein_couple recepto...
4,30289386,2018.0,simulation of spontaneous g protein activati...,g protein ; markov state model ; allostery ;...,[' washington university'],simulation of spontaneous g_protein activation...


In [24]:
df[['Id','fourgram']].to_csv('./Processed_data/Id_fourgram.csv')